The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:
!git clone https://github.com/d2l-ai/d2l-zh.git
import sys
sys.path.append('/content/d2l-zh')

Cloning into 'd2l-zh'...
remote: Enumerating objects: 24068, done.
remote: Total 24068 (delta 0), reused 0 (delta 0), pack-reused 24068 (from 1)
Receiving objects: 100% (24068/24068), 309.44 MiB | 21.22 MiB/s, done.
Resolving deltas: 100% (16835/16835), done.
Updating files: 100% (834/834), done.


# 用于预训练BERT的数据集
:label:`sec_bert-dataset`

为了预训练 :numref:`sec_bert`中实现的BERT模型，我们需要以理想的格式生成数据集，以便于两个预训练任务：遮蔽语言模型和下一句预测。一方面，最初的BERT模型是在两个庞大的图书语料库和英语维基百科（参见 :numref:`subsec_bert_pretraining_tasks`）的合集上预训练的，但它很难吸引这本书的大多数读者。另一方面，现成的预训练BERT模型可能不适合医学等特定领域的应用。因此，在定制的数据集上对BERT进行预训练变得越来越流行。为了方便BERT预训练的演示，我们使用了较小的语料库WikiText-2 :cite:`Merity.Xiong.Bradbury.ea.2016`。

与 :numref:`sec_word2vec_data`中用于预训练word2vec的PTB数据集相比，WikiText-2（1）保留了原来的标点符号，适合于下一句预测；（2）保留了原来的大小写和数字；（3）大了一倍以上。


In [2]:
import os
import random
import torch
from d2l import torch as d2l

在WikiText-2数据集中，每行代表一个段落，其中在任意标点符号及其前面的词元之间插入空格。保留至少有两句话的段落。为了简单起见，我们仅使用句号作为分隔符来拆分句子。我们将更复杂的句子拆分技术的讨论留在本节末尾的练习中。


In [4]:

#@save
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')

#@save
def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # 大写字母转换为小写字母
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs
"""
paragraphs是一个列表，元素是列表。类似：
 [
    ['sentence1', 'sentence2', 'sentence3', ...],
    ['sentence1', 'sentence2', ...],
    ...
]
"""

"\nparagraphs是一个列表，元素是列表。类似：\n [\n    ['sentence1', 'sentence2', 'sentence3', ...],\n    ['sentence1', 'sentence2', ...],\n    ...\n]\n"

## 为预训练任务定义辅助函数

在下文中，我们首先为BERT的两个预训练任务实现辅助函数。这些辅助函数将在稍后将原始文本语料库转换为理想格式的数据集时调用，以预训练BERT。

### 生成下一句预测任务的数据

根据 :numref:`subsec_nsp`的描述，`_get_next_sentence`函数生成二分类任务的训练样本。


In [5]:
#@save
# 有50%概率把第二个句子换成非next的句子。
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # paragraphs是三重列表的嵌套
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next
# 调用示例：
"""
paragraphs = [
    ["the cat sat on the mat", "it was sleepy"],
    ["dogs are loyal", "they love humans"]
]
_get_next_sentence("the cat sat on the mat", "it was sleepy", paragraphs)
返回的结果是：
('the cat sat on the mat', 'they love humans', False) 或者
('the cat sat on the mat', 'it was sleepy', True)
"""

'\nparagraphs = [\n    ["the cat sat on the mat", "it was sleepy"],\n    ["dogs are loyal", "they love humans"]\n]\n_get_next_sentence("the cat sat on the mat", "it was sleepy", paragraphs)\n返回的结果是：\n(\'the cat sat on the mat\', \'they love humans\', False) 或者\n(\'the cat sat on the mat\', \'it was sleepy\', True)\n'

下面的函数通过调用`_get_next_sentence`函数从输入`paragraph`生成用于下一句预测的训练样本。这里`paragraph`是句子列表，其中每个句子都是词元列表。自变量`max_len`指定预训练期间的BERT输入序列的最大长度。


In [6]:

#@save
"""
paragraph: 当前段落（句子列表）。形如 [['i', 'like', 'nlp'], ['it', 'is', 'fun'], ...]。
paragraphs: 所有段落的集合（用于在“负样本”情形下随机抽一句作为“伪下一句”）。
vocab: 词表对象（这里传入，但本函数内未使用；后续把 tokens 映射成索引时会用到）
max_len: 模型允许的总序列长度上限（含特殊符号）
这个代码就是把某一个段落的句子对都变成数据。
"""
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # 考虑1个''词元和2个''词元
        # 如果太长了就过滤掉
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        # tokens列表长度是tokena和tokenb的长度再加1个cls和2个sep
        # ex：tokens = ['','i','like','nlp','','it','is','fun','']
        # ex:segmenst = [0,0,0,0,0,1,1,1,1] 第一个cls和第一个sep都标0，第二个sep标1
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

### 生成遮蔽语言模型任务的数据
:label:`subsec_prepare_mlm_data`

为了从BERT输入序列生成遮蔽语言模型的训练样本，我们定义了以下`_replace_mlm_tokens`函数。在其输入中，`tokens`是表示BERT输入序列的词元的列表，`candidate_pred_positions`是不包括特殊词元的BERT输入序列的词元索引的列表（特殊词元在遮蔽语言模型任务中不被预测），以及`num_mlm_preds`指示预测的数量（选择15%要预测的随机词元）。在 :numref:`subsec_mlm`中定义遮蔽语言模型任务之后，在每个预测位置，输入可以由特殊的“掩码”词元或随机词元替换，或者保持不变。最后，该函数返回可能替换后的输入词元、发生预测的词元索引和这些预测的标签。


In [ ]:
#@save
"""
1.tokens比如['<cls>', 'i', 'like', 'nlp', '<sep>', 'it', 'is', 'fun', '<sep>']
2.candidate_pred_positions:可被遮蔽的token的位置索引列表（一般不包括 <cls> 和 <sep>）。
例如 [1, 2, 3, 5, 6, 7]
3.num_mlm_preds:要预测的 token 总数（通常是总 token 数的 15%）
4.vocab:词表对象，提供 vocab.idx_to_token（一个包含所有词的列表），用于随机替换
"""
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
                        vocab):
    # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的“<mask>”或随机词元
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        # 选择里面的num_mlm_preds个token进行mask
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10%的时间：保持词不变
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10%的时间：用随机词替换该词
            else:
                masked_token = random.choice(vocab.idx_to_token)
        # 让num_mlm_preds个需要被mask的token有80%概率被mask，10%概率是随机词，10%概率不变。
        mlm_input_tokens[mlm_pred_position] = masked_token
        # pred_positions_and_labels是列表，长度num_mlm_preds是元素是tuple，（需要被mask的token的position，变化后的token的string）
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels #第一个与原本的token是长度一致循序一致但是部分此被mask或者替换
    # 第二个是一个长度为num_mlm_preds的列表，元素是tuple（需要被mask的token的position，原本的token的string 类似标签 真实值）
    """
    ex：
    mlm_input_tokens = ['<cls>', 'i', '<mask>', 'nlp', '<sep>', 'it', 'is', '<mask>', '<sep>']
    pred_positions_and_labels = [(2, 'like'), (7, 'fun')]
    """

通过调用前述的`_replace_mlm_tokens`函数，以下函数将BERT输入序列（`tokens`）作为输入，并返回输入词元的索引（在 :numref:`subsec_mlm`中描述的可能的词元替换之后）、发生预测的词元索引以及这些预测的标签索引。


In [ ]:
#@save
"""
tokens: 当前句子（或句子对）的 token 列表，比如['<cls>', 'i', 'like', 'nlp', '<sep>', 'it', 'is', 'fun', '<sep>']
与前面的函数的参数tokens一样。
这是这个函数是实现了根据输入tokens得到candidate_pred_positions然后调用前面的_replace_mlm_tokens
最终返回：
"""

"""
示例输入
tokens = ['<cls>', 'i', 'like', 'nlp', '<sep>', 'it', 'is', 'fun', '<sep>']
示例输出
vocab[mlm_input_tokens]=[...]长度与原本的tokens一致元素的token元素对应的idx （其中，mlm_input_tokens = ['<cls>', 'i', '<mask>', 'nlp', '<sep>', 'it', 'is', '<mask>', '<sep>']
pred_positions = [2, 7]
vocab[mlm_pred_labels] = [...]长度与pred_positions一致，元素是idx （其中，mlm_pred_labels = ['like', 'fun']
"""
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # tokens是一个字符串列表
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i) #构造 需要去除cls、sep的token列表
    # 遮蔽语言模型任务中预测15%的随机词元
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels] # 真正被mask或者替换或未变的那15%的token的位置
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels] # 真实值，原本的token
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

## 将文本转换为预训练数据集

现在我们几乎准备好为BERT预训练定制一个`Dataset`类。在此之前，我们仍然需要定义辅助函数`_pad_bert_inputs`来将特殊的“&lt;mask&gt;”词元附加到输入。它的参数`examples`包含来自两个预训练任务的辅助函数`_get_nsp_data_from_paragraph`和`_get_mlm_data_from_tokens`的输出。


In [ ]:
#@save
"""
examples是列表，元素是元组。即(token_ids, pred_positions, mlm_pred_label_ids, segments, is_next) 长度由什么决定？
其中
来自mlm的：
token_ids 是列表，句子A+B的词元id序列（含 [CLS], [SEP]）
pred_positions	被mask的token索引
mlm_pred_label_ids	被mask位置的真实标签
来自nsp的：
segments列表，长度与tokens一致	元素：（句子A=0, 句子B=1）
is_next	int类型，	NSP标签（1=是下一句，0=不是）

max_len 最大句子长度（统一 padding 长度）
vocab 词表对象
"""
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15) #需要被mask的词的个数
    """
    all_token_ids	(batch_size, max_len)	padding 后的 token 序列
    all_segments	(batch_size, max_len)	padding后的segment 序列
    valid_lens	(batch_size,)	每个句子的真实长度
    all_pred_positions	(batch_size, max_num_mlm_preds)	MLM mask 位置
    all_mlm_weights	(batch_size, max_num_mlm_preds)	MLM 权重（0/1）
    all_mlm_labels	(batch_size, max_num_mlm_preds)	MLM 标签
    nsp_labels	(batch_size,)	NSP标签（0/1）
    """
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for (token_ids, pred_positions, mlm_pred_label_ids, segments,
         is_next) in examples:
        # 把tokens_ids补成长度都是max len。从而统一所有样本的token_ids长度
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
            max_len - len(token_ids)), dtype=torch.long))
        # 其余部分补0（后续可能需要valid len来解决这个补充的0？）
        all_segments.append(torch.tensor(segments + [0] * (
            max_len - len(segments)), dtype=torch.long))
        # valid_lens不包括'<pad>'的计数 ，列表 长度是(样本数)
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        # 把pred_positions补成长度是max lenth
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
            max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 填充词元的预测将通过乘以0权重在损失中过滤掉

        """
        EX：
        mlm_pred_label_ids = [1037, 2054, 102]
        max_num_mlm_preds = 6
        all_mlm_weights = [1, 1, 1, 0, 0, 0]
        """
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32))
        """
        EX：
        mlm_pred_label_ids = [1037, 2054, 102]
        max_num_mlm_preds = 6
        则 all_mlm_labels = [1037, 2054, 102, 0, 0, 0]
        """
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
            max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

将用于生成两个预训练任务的训练样本的辅助函数和用于填充输入的辅助函数放在一起，我们定义以下`_WikiTextDataset`类为用于预训练BERT的WikiText-2数据集。通过实现`__getitem__ `函数，我们可以任意访问WikiText-2语料库的一对句子生成的预训练样本（遮蔽语言模型和下一句预测）样本。

最初的BERT模型使用词表大小为30000的WordPiece嵌入 :cite:`Wu.Schuster.Chen.ea.2016`。WordPiece的词元化方法是对 :numref:`subsec_Byte_Pair_Encoding`中原有的字节对编码算法稍作修改。为简单起见，我们使用`d2l.tokenize`函数进行词元化。出现次数少于5次的不频繁词元将被过滤掉。


In [ ]:
#@save
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 输入paragraphs[i]是代表段落的句子字符串列表；
        # 而输出paragraphs[i]是代表段落的句子列表，其中每个句子都是词元列表
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])
        # 获取下一句子预测任务的数据
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        # 获取遮蔽语言模型任务的数据
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                      + (segments, is_next))
                     for tokens, segments, is_next in examples]
        # 填充输入
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

通过使用`_read_wiki`函数和`_WikiTextDataset`类，我们定义了下面的`load_data_wiki`来下载并生成WikiText-2数据集，并从中生成预训练样本。


In [ ]:
#@save
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                        shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

将批量大小设置为512，将BERT输入序列的最大长度设置为64，我们打印出小批量的BERT预训练样本的形状。注意，在每个BERT输入序列中，为遮蔽语言模型任务预测$10$（$64 \times 0.15$）个位置。


In [ ]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break

torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])


最后，我们来看一下词量。即使在过滤掉不频繁的词元之后，它仍然比PTB数据集的大两倍以上。


In [ ]:
len(vocab)

20256

## 小结

* 与PTB数据集相比，WikiText-2数据集保留了原来的标点符号、大小写和数字，并且比PTB数据集大了两倍多。
* 我们可以任意访问从WikiText-2语料库中的一对句子生成的预训练（遮蔽语言模型和下一句预测）样本。

## 练习

1. 为简单起见，句号用作拆分句子的唯一分隔符。尝试其他的句子拆分技术，比如Spacy和NLTK。以NLTK为例，需要先安装NLTK：`pip install nltk`。在代码中先`import nltk`。然后下载Punkt语句词元分析器：`nltk.download('punkt')`。要拆分句子，比如`sentences = 'This is great ! Why not ?'`，调用`nltk.tokenize.sent_tokenize(sentences)`将返回两个句子字符串的列表：`['This is great !', 'Why not ?']`。
1. 如果我们不过滤出一些不常见的词元，词量会有多大？


[Discussions](https://discuss.d2l.ai/t/5738)
